In [83]:
%load_ext autoreload
%autoreload 2


from src.model.config import CONFIG_IN_IMPLEMENTATION, CONFIG_IN_PAPER
from src.model.pprec import PPRec
from src.data.train import EBNeRDTrainDataset


import torch
from torch.utils.data import DataLoader
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Some data for testing


In [84]:
dataset = EBNeRDTrainDataset()

Configuration of the model

In [85]:
max_clicked = 50

model = PPRec(
    max_clicked=max_clicked,
    device=torch.device("cpu"),
    config=CONFIG_IN_IMPLEMENTATION,
    # config=CONFIG_IN_PAPER
)

In [86]:
batch_size = 32
candidate_size = 4

candidates = PPRec.CandidateBatch(
    ids=np.array(
        [
            [dataset.split.get_random_article_id() for _ in range(candidate_size)]
            for _ in range(batch_size)
        ]
    ),
    ctr=torch.rand(batch_size, candidate_size),
    recencies=torch.rand(batch_size, candidate_size),
)

user_clicks = PPRec.ClicksBatch(
    ids=np.array(
        [
            [dataset.split.get_random_article_id() for _ in range(max_clicked)]
            for _ in range(batch_size)
        ]
    ),
    ctr=torch.rand(batch_size, max_clicked),
)

In [87]:
# Works with random inputs!
result = model(PPRec.Inputs(clicks=user_clicks, candidates=candidates))
result.personalized_matching_score.shape, result.popularity_score.shape, result.score.shape

(torch.Size([32, 4]), torch.Size([32, 4]), torch.Size([32, 4]))

In [88]:
# Just gotta finish the dataset in src.models.train
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=dataset.collate_fn)

In [89]:
sum(p.numel() for p in model.parameters() if p.requires_grad)

1629116

In [90]:
for batch in dataloader:
    inputs = model.preprocess_train_batch(batch)
    predictions = model(inputs)
    break